In [134]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import json

# 주가 정리

In [139]:
price_list[i]

,시가,고가,저가,종가,거래량,외국인소진율
날짜,,,,,,
20170626,178453,180856,176450,177852,58948,61.28
20170627,178250,180654,173445,174046,686346,61.22
20170628,171842,173244,170440,172444,101311,61.15
20170629,173045,173846,170441,170842,72186,61.12
20170630,169240,170241,167237,167838,111979,61.12
...,...,...,...,...,...,...
20220620,239500,241000,230000,234000,719666,53.52
20220621,236000,241500,234500,239500,526317,53.56
20220622,239500,239500,227000,229000,894578,53.53


In [135]:
KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].apply(lambda x: '{0:06d}'.format(x))
KOR_ticker.head()

price_list = []
for i in tqdm(range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    code = KOR_ticker['종목코드'][i]
    price_list.append(pd.read_csv('data/KOR_price/{}_price.csv'.format(code), encoding='cp949'))
    price_list[i] = price_list[i].set_index('날짜')
    price_list[i].columns = ['{}'.format(code)]

price_list = pd.concat(price_list, axis=1)
price_list.reset_index(drop=False, inplace=True)
price_list.to_csv('data/KOR_price.csv', encoding='cp949', index=False)

  0%|            | 5/2268 [00:00<00:42, 53.76it/s]


ValueError: Length mismatch: Expected axis has 6 elements, new values have 1 elements

# 재무제표 정리

In [140]:
# 개별 재무재표 불러오기
KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].apply(lambda x: '{0:06d}'.format(x))

data_fs = []
for i in tqdm(range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    code = KOR_ticker['종목코드'][i]
    data_fs.append(pd.read_csv('data/KOR_fs/{}_fs.csv'.format(code), encoding='cp949').set_index('계정'))
    if i == 4:
        break

fs_items = data_fs[0].index.tolist()
print(len(fs_items))
print(fs_items[:5])

  0%|            | 4/2268 [00:00<00:36, 62.49it/s]

51
['매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익']


In [86]:
# # 데이터 정리
# fs_dict = defaultdict(lambda: pd.DataFrame())
# for account in fs_items:
#     for i in range(len(data_fs)):
#         code = KOR_ticker['종목코드'][i]
#         data = data_fs[i].loc[account, :].reset_index().rename(columns={'index': 'year'})
#         data = pd.DataFrame([data[account].tolist()], columns=data['year'].tolist(), index=[code])
#         fs_dict[account] = pd.concat([fs_dict[account], data])
#     break
# fs_dict[account]

# save
fs_dict = defaultdict(dict)
for account in fs_items:
    for i in range(len(data_fs)):
        code = KOR_ticker['종목코드'][i]
        data = data_fs[i].loc[account, :].reset_index().rename(columns={'index': 'year'})
        data = {year: amount for year, amount in zip(data['year'], data[account])}
        fs_dict[account][code] = data
with open("data/KOR_fs.json", "w") as json_file:
    json.dump(fs_dict, json_file)

In [88]:
# load
with open("data/KOR_fs.json", "r") as f:
    data = json.load(f)

fs_data = {}
for account, _data in data.items():
    fs_data[account] = pd.DataFrame(_data).transpose()

{'매출액': {'005930': {'2019/12': 2304009.0,
   '2020/12': 2368070.0,
   '2021/12': 2796048.0},
  '373220': {'2019/12': nan, '2020/12': 14611.0, '2021/12': 178519.0},
  '000660': {'2019/12': 269907.0, '2020/12': 319004.0, '2021/12': 429978.0},
  '207940': {'2019/12': 7016.0, '2020/12': 11648.0, '2021/12': 15680.0},
  '051910': {'2019/12': 273531.0, '2020/12': 300589.0, '2021/12': 426547.0}},
 '매출원가': {'005930': {'2019/12': 1472395.0,
   '2020/12': 1444883.0,
   '2021/12': 1664113.0},
  '373220': {'2019/12': nan, '2020/12': 12370.0, '2021/12': 139531.0},
  '000660': {'2019/12': 188188.0, '2020/12': 210898.0, '2021/12': 240456.0},
  '207940': {'2019/12': 5096.0, '2020/12': 7487.0, '2021/12': 8416.0},
  '051910': {'2019/12': 226674.0, '2020/12': 237414.0, '2021/12': 314364.0}},
 '매출총이익': {'005930': {'2019/12': 831613.0,
   '2020/12': 923187.0,
   '2021/12': 1131935.0},
  '373220': {'2019/12': nan, '2020/12': 2240.0, '2021/12': 38988.0},
  '000660': {'2019/12': 81719.0, '2020/12': 108106.0, '

# 가치지표 정리

In [153]:
# 개별 가치지표 불러오기
KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].apply(lambda x: '{0:06d}'.format(x))

data_value = []
not_loaded = []
for i in tqdm(range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    try:
        code = KOR_ticker['종목코드'][i]
        _data = pd.read_csv('data/KOR_value/{}_value.csv'.format(code), encoding='cp949')
        _data['종목코드'] = [code]
        _data.set_index('종목코드', inplace=True)
        data_value.append(_data)
    except FileNotFoundError:
        not_loaded.append(code)

data_value = pd.concat(data_value)
data_value.reset_index(inplace=True)
data_value.to_csv('data/KOR_value.csv', encoding='cp949', index=False)

100%|████████| 2268/2268 [00:22<00:00, 100.19it/s]


In [155]:
data_value.to_csv('data/KOR_value.csv', encoding='cp949', index=False)